## load data

In [70]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import math
import matplotlib.pyplot as plt
# Read in data
def loadData(filename,path="ml-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split('\t')
            data.append({ "user_id": str(user), "movie_id": str(movieid)})
            y.append(float(rating))
            users.add(user)
            items.add(movieid)

    return (data, np.array(y), users, items)

In [71]:
(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")

# (num_user,num_mv) = np.amax(train_data, axis=0)
# N = len(train_data)

v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

In [74]:
print X_train.shape

(90570, 2623)


## SGD

目标函数
![](img_fm/1.png)
加入正则项，防止过拟合
![](img_fm/2.png)
Algorithm for SGD
![](img_fm/3.png)
![](img_fm/4.png)

In [77]:
# X = np.ones((N, 3))
# X[:,1:] = train_data
n = X_train.shape[1]
k = 10

w0 = np.random.rand(1)  # bias
W = np.random.rand(1,n)
V = np.random.rand(n,k)
learning_rate = 0.0001

loss_sgd = []

lamb = 0.1 # use global regularization first

In [83]:
X_train[1,:]

<1x2623 sparse matrix of type '<type 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [84]:
import math
for i in range(1):
    
    # calculate train error
    
    y_predict = []
    for m in range(N):
        fm_part = 0
        for j in range(n-1):
            for k in range(j+1,n):
                fm_part += V[j,:].dot(V[k,:].T)*X_train[m,j]*X_train[m,k]

        y_predict.append(w0 + np.sum(W.dot(X[m,:])) + fm_part)

    y_predict = np.array(y_predict).T
    loss = np.sum(np.square(y_predict - y)) + np.sum(np.square(W)) + w0*w0 + np.sum(np.square(V))
    loss = math.sqrt(loss / N)
    
    print "Iter " , k  , "\t" + "loss: " , loss
    
    # calculate train error end
    
    # update params
    w0 = w0 - learning_rate*1
    W = W - learning_rate*X_train[i,:]
    
    
#     w0 = w0 - learning_rate * 
#     W = W - learning_rate * X[i,:] + 2 * lambda * W

TypeError: list indices must be integers, not tuple

In [59]:
y_predict

array([[  4.78778802e+00],
       [  8.54776317e+00],
       [  1.23077383e+01],
       ..., 
       [  3.75269012e+06],
       [  3.87902984e+06],
       [  4.20119613e+06]])

In [65]:
num_user + num_mv

2625